In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
dataset_train = pd.read_csv("trainS&PSL20.csv")

In [5]:
# Specify the columns you want to remove commas from
columns_to_clean = [,'Open', 'High']

# Loop through the specified columns and remove commas
for column in columns_to_clean:
    dataset_train[column] = dataset_train[column].str.replace(',', '', regex=True)

# Save the cleaned dataset back to a new file
#cleaned_filename = 'cleaned_data.csv'
#dataset.to_csv(cleaned_filename, index=False)


In [6]:
dataset_train.head()

,Date,Price,Open,High,Low,Change %
0,12/09/2021,"3,985.89",3959.84,3998.37,"3,950.84",0.66%
1,12/08/2021,"3,959.84",3964.42,3986.86,"3,950.75",-0.12%
2,12/07/2021,"3,964.42",3947.00,3990.29,"3,947.00",0.44%
3,12/06/2021,"3,947.00",3884.57,3968.44,"3,884.57",1.61%
4,12/03/2021,"3,884.57",3912.88,3913.67,"3,854.18",-0.72%


In [7]:
dataset_train.isna().sum()

Date        0
Price       0
Open        0
High        0
Low         0
Change %    0
dtype: int64

In [9]:
training_set = dataset_train.iloc[:, [0,2,3]].values

In [11]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

ValueError: ignored

In [ ]:
X_train = []
y_train = []

for i in range(60, 2035):
  X_train.append(training_set_scaled[i-60:i, 0])
  y_train.append(training_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

In [ ]:
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X_train,y_train,epochs=100,batch_size=32)

In [ ]:
dataset_test = pd.read_csv("testS&PSL20.csv")

In [ ]:
# Specify the columns you want to remove commas from
columns_to_clean = ['Open', 'High']

# Loop through the specified columns and remove commas
for column in columns_to_clean:
    dataset_test[column] = dataset_test[column].str.replace(',', '', regex=True)

In [ ]:

real_stock_price = dataset_test.iloc[:, 0,2:3].values

In [ ]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []

for i in range(60, 76):
  X_test.append(inputs[i-60:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
# Convert numpy arrays to float
real_stock_price = real_stock_price.astype(float)
predicted_stock_price = predicted_stock_price.astype(float)

In [ ]:
plt.plot(real_stock_price, color = 'black', label = 'S&P SL20')
plt.plot(predicted_stock_price, color = 'green', label = 'Predicted S@P SL20 Price')
plt.title('S&P SL20 Price Prediction')
plt.xlabel('Time')
plt.ylabel('S&P SL20 Price')
plt.legend()
plt.show()